In [1]:
#### DO NOT RUN THIS CODE IN THE NOTEBOOK - copy paste into terminal in the HOME directory!

# git clone https://github.com/tensorflow/models.git

In [14]:
 # !pip install s3fs -q
 # !pip install tensorflow_hub -q
 # !pip install tensorflow_datasets -q
 # !pip install pycocotools -q
 # !pip install gin-config -q
 # !pip install immutabledict -q
 # !pip install sentencepiece -q
 # !pip install transformers -q
 # !pip install -r official/projects/movinet/requirements.txt -q

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import configparser

config = configparser.ConfigParser()
config.read('credentials')

['credentials']

In [3]:
# AWS credentials and S3 settings
aws_access_key_id = config["default"]['aws_access_key_id']
aws_secret_access_key = config["default"]['aws_secret_access_key']
bucket_name = 'asl-capstone'
s3_URI = 's3://asl-capstone/'

In [4]:
import s3fs
fs = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key)

import boto3
s3 = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key,
                  region_name = 'us-west-2')


In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

tf.keras.backend.clear_session()

2023-11-08 04:29:13.006069: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-08 04:29:13.852072: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
%cd /home/ec2-user/models

/home/ec2-user/models


In [7]:
pwd

'/home/ec2-user/models'

In [8]:
# !wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a0_base.tar.gz
# !wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a3_base.tar.gz
# !wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a5_base.tar.gz
# !wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a5_base_k400.tar.gz

# !tar -xf movinet_a5_base_k400.tar.gz
# !mv movinet_a5_base movinet_a5_base_k400
# !tar -xf movinet_a0_base.tar.gz
# !tar -xf movinet_a3_base.tar.gz
# !tar -xf movinet_a5_base.tar.gz

In [9]:
model_version = 'a3'

if model_version=='a3':
  max_frames = 120
  image_dims = (256,256)
elif model_version=='a0':
  max_frames = 50
  image_dims = (172,172)
elif model_version=='a5':
  max_frames = 120
  image_dims = (320,302)

In [10]:
# Get the kinetics-600 action labels
KINETICS_URL = "official/projects/movinet/files/kinetics_600_labels.txt"
with open(KINETICS_URL) as obj:
  labels_600 = [line for line in obj.readlines()]
print("Found %d labels." % len(labels_600))

Found 600 labels.


In [11]:
# # Get the kinetics-400 action labels from the GitHub repository.
# from urllib import request  # requires python3
# KINETICS_URL = "https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map.txt"
# with request.urlopen(KINETICS_URL) as obj:
#   labels_400 = [line.decode("utf-8").strip() for line in obj.readlines()]
# print("Found %d labels." % len(labels_400))

In [12]:
# !export XLA_PYTHON_CLIENT_PREALLOCATE=false
# !export XLA_FLAGS="--xla_gpu_strict_conv_algorithm_picker=false --xla_gpu_force_compilation_parallelism=1"

In [13]:
from official.projects.movinet.modeling import movinet_model

from official.projects.movinet.modeling import movinet

# Create backbone and model.
backbone = movinet.Movinet(
    model_id=model_version, #change to correspond to model
    causal=False,
    use_external_states=False,
)

model = movinet_model.MovinetClassifier(
    backbone, num_classes=600, output_states=False) #change numclasses for dataset

# Create your example input here.
# Refer to the paper for recommended input shapes.
inputs = tf.ones([1, 120, image_dims[0], image_dims[1], 3]) #make sure input shape corresponds to correct model

# Build the model
model.build(inputs.shape)

# load pretrained checkpoint
checkpoint_dir = f'movinet_{model_version}_base'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
status = checkpoint.restore(checkpoint_path)
status.assert_existing_objects_matched()



2023-11-08 04:29:25.818222: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [14]:
def pad_video_frames(video, n_frames=120):
  if video.shape[0] < n_frames:
    video = np.concatenate([video,np.zeros((n_frames - video.shape[0], 256, 256, 3))])
    # video_tf = tf.convert_to_tensor(video, dtype=tf.float32)
    return video
  elif video.shape[0] > n_frames:
    video = video[np.round(np.linspace(0, len(video)-1, num=n_frames)).astype(int)]
    # video_tf = tf.convert_to_tensor(video, dtype=tf.float32)
    return video
  else:
    # video_tf = tf.convert_to_tensor(video, dtype=tf.float32)
    return video

In [15]:
video_uri = "s3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/06kKvQp4SfM_12.npy"
with fs.open(video_uri,'rb') as f:
    videos = np.load(f)

In [16]:
videos = np.expand_dims(videos, axis = 0)
print(videos.shape)

(1, 74, 256, 256, 3)


In [17]:
# Run the model prediction.
output = model(videos)
prediction = tf.argmax(output, -1)
    
print(labels_600[prediction.numpy()[0]])
#print(labels_600[prediction.numpy()[1]])

2023-11-08 04:30:31.351751: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xa8dc340 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-11-08 04:30:31.351781: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-11-08 04:30:31.361239: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


sign language interpreting



In [18]:
video_uri = "s3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/06kKvQp4SfM_12.npy"
with fs.open(video_uri,'rb') as f:
    videos = np.load(f)
videos = pad_video_frames(videos)
print(videos.shape)
videos = np.expand_dims(videos, axis = 0)
print(videos.shape)

(120, 256, 256, 3)
(1, 120, 256, 256, 3)


In [19]:
# Run the model prediction.
output = model(videos)
prediction = tf.argmax(output, -1)
    
print(labels_600[prediction.numpy()[0]])
#print(labels_600[prediction.numpy()[1]])

sign language interpreting



In [20]:
video_uri = "s3://asl-capstone/wlasl/book_3.npy"
with fs.open(video_uri,'rb') as f:
    videos = np.load(f)
videos = np.expand_dims(videos, axis = 0)
print(videos.shape)
# Run the model prediction.
output = model(videos)
prediction = tf.argmax(output, -1)
print("Prediction")
print(labels_600[prediction.numpy()[0]])


(1, 86, 256, 256, 3)
Prediction
sign language interpreting



In [21]:
# Pick 20 files at random from YouTube ASL and run it through MovieNet
bucket_path = 'youtube-asl/test_sample/numpy_files/RGB/'
# List all files in the S3 path
files = fs.ls(f'{bucket_name}/{bucket_path}')
print(len(files))

559


In [22]:
video_files = [file for file in files if "_cap.npy" not in file]

In [23]:
import random
video_files = random.sample(video_files, 20)

In [25]:
video_files = ["s3://"+x for x in video_files]
print(video_files)

['s3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/06kKvQp4SfM_11.npy', 's3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/Dmvi3_q3gMc_110.npy', 's3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/5J8a2CaXWvk_88.npy', 's3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/06kKvQp4SfM_15.npy', 's3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/5J8a2CaXWvk_8.npy', 's3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/Dmvi3_q3gMc_71.npy', 's3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/Dmvi3_q3gMc_76.npy', 's3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/0XGfpv6PUq4_24.npy', 's3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/5J8a2CaXWvk_65.npy', 's3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/Dmvi3_q3gMc_93.npy', 's3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/5J8a2CaXWvk_110.npy', 's3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/oOkSSJbNv68_28.npy', 's3://asl-capstone/youtube-asl/test_sa

In [27]:
counter = 1
for video_uri in video_files:
    with fs.open(video_uri,'rb') as f:
        videos = np.load(f)
    videos = np.expand_dims(videos, axis = 0)
    print(videos.shape)
    # Run the model prediction.
    output = model(videos)
    prediction = tf.argmax(output, -1)
    print(f"Prediction for {counter} video")
    print("Prediction: ",labels_600[prediction.numpy()[0]])
    counter+=1
    #print(labels_600[prediction.numpy()[0]])

(1, 63, 256, 256, 3)
Prediction for 1 video
Prediction:  sign language interpreting

(1, 186, 256, 256, 3)
Prediction for 2 video
Prediction:  sign language interpreting

(1, 44, 256, 256, 3)
Prediction for 3 video
Prediction:  sign language interpreting

(1, 110, 256, 256, 3)
Prediction for 4 video
Prediction:  sign language interpreting

(1, 82, 256, 256, 3)
Prediction for 5 video
Prediction:  sign language interpreting

(1, 245, 256, 256, 3)
Prediction for 6 video
Prediction:  sign language interpreting

(1, 57, 256, 256, 3)
Prediction for 7 video
Prediction:  sign language interpreting

(1, 210, 256, 256, 3)
Prediction for 8 video
Prediction:  sign language interpreting

(1, 191, 256, 256, 3)
Prediction for 9 video
Prediction:  sign language interpreting

(1, 160, 256, 256, 3)
Prediction for 10 video
Prediction:  sign language interpreting

(1, 108, 256, 256, 3)
Prediction for 11 video
Prediction:  sign language interpreting

(1, 182, 256, 256, 3)
Prediction for 12 video
Prediction

ValueError: Input 0 of layer "movinet_classifier" is incompatible with the layer: expected shape=(None, None, None, None, 3), found shape=(1, 50, 120, 256, 256, 3)